<a href="https://colab.research.google.com/github/mzhasan00/pytorch-deep-learning/blob/main/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [2]:
# YOUR CODE HERE
%%writefile get_data.py

import requests
import zipfile
from pathlib import Path

data_dir = Path("data/")
image_path = data_dir / "pizza_steak_sushi"

if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

with open(data_dir / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

with zipfile.ZipFile(data_dir / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...")
    zip_ref.extractall(image_path)

Writing get_data.py


In [3]:
# Example running of get_data.py
!python get_data.py

Did not find data/pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

In [12]:
!rm -rf going_modular

In [13]:
# prompt: load going_modular folder

!git clone https://github.com/mrdbourke/pytorch-deep-learning.git
!mv pytorch-deep-learning/going_modular/going_modular .
!rm -rf pytorch-deep-learning


Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4177, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 4177 (delta 61), reused 104 (delta 38), pack-reused 4035 (from 1)
Receiving objects: 100% (4177/4177), 651.42 MiB | 31.62 MiB/s, done.
Resolving deltas: 100% (2432/2432), done.
Updating files: 100% (248/248), done.


In [32]:
# prompt: write code for the description above
%%writefile train.py

import os
import argparse
import torch
from torchvision import transforms
from going_modular.model_builder import TinyVGG
from going_modular.data_setup import create_dataloaders
from going_modular.engine import train_step, test_step

device = "cuda" if torch.cuda.is_available() else "cpu"

# Setup hyperparameters
parser = argparse.ArgumentParser(description="Get hyperparameters for training")
parser.add_argument("--train_dir", default="data/pizza_steak_sushi/train", type=str, help="Training directory")
parser.add_argument("--test_dir", default="data/pizza_steak_sushi/test", type=str, help="Testing directory")
parser.add_argument("--learning_rate", default=0.001, type=float, help="Learning rate")
parser.add_argument("--batch_size", default=32, type=int, help="Batch size")
parser.add_argument("--num_epochs", default=5, type=int, help="Number of epochs to train for")
parser.add_argument("--hidden_units", default=10, type=int, help="Number of hidden units in TinyVGG model")
args = parser.parse_args()


data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

model = TinyVGG(input_shape=3, hidden_units=args.hidden_units, output_shape=3).to(device)
train_dataloader, test_dataloader, class_names = create_dataloaders(args.train_dir, args.test_dir, data_transform, args.batch_size)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

result = {
    "train_loss": [],
  	"train_acc": [],
  	"test_loss": [],
  	"test_acc": []
}

for epoch in range(args.num_epochs):
  print(f"Epoch: {epoch}\n---------")
  train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device)
  test_loss, test_acc = test_step(model, test_dataloader, loss_fn, device)

  result["train_loss"].append(train_loss)
  result["train_acc"].append(train_acc)
  result["test_loss"].append(test_loss)
  result["test_acc"].append(test_acc)

  print(f"Train loss: {train_loss:.2f}")
  print(f"Test loss: {test_loss:.2f}")
  print(f"Train acc: {train_acc:.2f}%")
  print(f"Test acc: {test_acc:.2f}%")
  print("\n")

# Save the model
os.makedirs('model', exist_ok=True)
torch.save(model.state_dict(), "model/tinyvgg.pth")

Overwriting train.py


In [33]:
# Example running of train.py
!python train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

Epoch: 0
---------
Train loss: 1.10
Test loss: 1.09
Train acc: 0.39%
Test acc: 0.33%


Epoch: 1
---------
Train loss: 1.08
Test loss: 1.08
Train acc: 0.35%
Test acc: 0.39%


Epoch: 2
---------
Train loss: 1.05
Test loss: 1.04
Train acc: 0.49%
Test acc: 0.52%


Epoch: 3
---------
Train loss: 1.00
Test loss: 1.00
Train acc: 0.55%
Test acc: 0.47%


Epoch: 4
---------
Train loss: 0.95
Test loss: 0.99
Train acc: 0.57%
Test acc: 0.48%


Traceback (most recent call last):
  File "/content/train.py", line 58, in <module>
    os.makedirs('model')
  File "/usr/lib/python3.10/os.py", line 225, in makedirs
    mkdir(name, mode)
FileExistsError: [Errno 17] File exists: 'model'


In [ ]:
from going_modular.model_builder import TinyVGG

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = TinyVGG(3, 10, 3).to(device)
model.load

## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [ ]:
# prompt: load state_dict to a TinyVGG model




In [34]:
from PIL import Image
import matplotlib.pyplot as plt
from going_modular.model_builder import TinyVGG

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = TinyVGG(3, 10, 3).to(device)
model.load_state_dict(torch.load("model/tinyvgg.pth"))

image = Image.open("data/pizza_steak_sushi/test/sushi/175783.jpg")
image = image.resize((64, 64))
image = torch.Tensor(image).permute(2, 0, 1)

logits = model(image)
pred_probs = torch.softmax(logits, dim=1)
pred_label = torch.argmax(pred_probs, dim=1)


plt.figure()
plt.imshow(image.squeeze().permute(1, 2, 0).cpu())
plt.title(f"Pred: {train_data.classes[pred_label]} | Prob: {pred_probs.max():.3f}")
plt.axis("off")


FileNotFoundError: [Errno 2] No such file or directory: 'model/tinyvgg.pth'

In [ ]:
# YOUR CODE HERE
%%writefile predict.py

from PIL import Image
import matplotlib.pyplot as plt

image = Image.open("data/pizza_steak_sushi/test/sushi/175783.jpg")
image = image.resize((64, 64))
image = torch.Tensor(image).permute(2, 0, 1)

logits = model(image)
pred_probs = torch.softmax(logits, dim=1)
pred_label = torch.argmax(pred_probs, dim=1)


plt.figure()
plt.imshow(image.squeeze().permute(1, 2, 0).cpu())
plt.title(f"Pred: {train_data.classes[pred_label]} | Prob: {pred_probs.max():.3f}")
plt.axis("off")


In [ ]:
# Example running of predict.py
!python predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg